In [ ]:
import sys
sys.path.append('../../build')
import libry as ry
import numpy as np
import time

In [ ]:
#-- REAL WORLD configuration, which is attached to the physics engine
# accessing this directly would be cheating!
RealWorld = ry.Config()
RealWorld.addFile("../../scenarios/pandasTable.g")

In [ ]:
S = RealWorld.simulation(ry.SimulatorEngine.bullet, True)
S.addSensor("camera")

In [ ]:
#-- MODEL WORLD configuration, this is the data structure on which you represent
# what you know about the world and compute things (controls, contacts, etc)
C = ry.Config()
#D = C.view() #rather use the ConfiguratioViewer below
C.addFile("../../scenarios/pandasTable.g")
C.view()

In [ ]:
# add a new frame to the MODEL configuration
# (Perception will later have to do exactly this: add perceived objects to the model)
obj = C.addFrame("object")

# set frame parameters, associate a shape to the frame, 
obj.setPosition([.8,0,1.0])
obj.setQuaternion([1,0,.5,0])
obj.setShape(ry.ST.ssBox, [.2,.01])
obj.setColor([1,0,1])

In [ ]:
obj.getPosition()
obj.getSize()
obj.getPosition()

C.getFrameNames()
gripper = C.getFrame('R_gripper')
gripper.getPosition()
C.getJointState()


In [ ]:
#-- the following is the simulation loop
tau = .01

for t in range(1000):
    time.sleep(0.01)

    #grab sensor readings from the simulation
    q = S.get_q()
    if t%10 == 0:
            [rgb, depth] = S.getImageAndDepth()  #we don't need images with 100Hz, rendering is slow
    #some good old fashioned IK
    C.setJointState(q) #set your robot model to match the real q
    obj.setPosition(obj.getPosition()+np.array([0,0,0.001]))

    [y,J] = C.evalFeature(ry.FS.positionDiff, ["L_gripper", "object"])
    vel = J.T @ np.linalg.inv(J@J.T + 1e-2*np.eye(y.shape[0])) @ (-y);

    #send velocity controls to the simulation
    S.step(vel, tau, ry.ControlMode.velocity)

In [ ]:
C.getFrameNames()
